Parameters

In [1]:
raw_data_path = '/content/drive/My Drive/github_repos/Fake-news-detection/Data/news.csv'
destination_folder = '/content/drive/My Drive/github_repos/Fake-news-detection/Data'

first_n_words = 200

Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

Preprocessing

In [3]:
def trim_string(x):

    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])

    return x

In [5]:
# Read raw data
df_raw = pd.read_csv(raw_data_path)

# Prepare columns
df_raw['label'] = (df_raw['label'] == 'FAKE').astype('int')
df_raw['titletext'] = df_raw['title'] + ". " + df_raw['text']
df_raw = df_raw.reindex(columns=['label', 'title', 'text', 'titletext'])

# Drop rows with empty text
df_raw.drop( df_raw[df_raw.text.str.len() < 5].index, inplace=True)

# Trim text and titletext to first_n_words
df_raw['text'] = df_raw['text'].apply(trim_string)
df_raw['titletext'] = df_raw['titletext'].apply(trim_string) 

# Split according to label
df_real = df_raw[df_raw['label'] == 0]
df_fake = df_raw[df_raw['label'] == 1]

# Train-test split
df_real_full_train, df_real_test = train_test_split(df_real, test_size = 0.1, random_state = 1)
df_fake_full_train, df_fake_test = train_test_split(df_fake, test_size = 0.1, random_state = 1)

# Train-valid split
df_real_train, df_real_valid = train_test_split(df_real_full_train, train_size = 0.89, random_state = 1)
df_fake_train, df_fake_valid = train_test_split(df_fake_full_train, train_size = 0.89, random_state = 1)

# Concatenate splits of different labels
df_train = pd.concat([df_real_train, df_fake_train], ignore_index=True, sort=False)
df_valid = pd.concat([df_real_valid, df_fake_valid], ignore_index=True, sort=False)
df_test = pd.concat([df_real_test, df_fake_test], ignore_index=True, sort=False)
print(df_train.shape, df_valid.shape, df_test.shape)

# Write preprocessed data
df_train.to_csv(destination_folder + '/train.csv', index=False)
df_valid.to_csv(destination_folder + '/valid.csv', index=False)
df_test.to_csv(destination_folder + '/test.csv', index=False)

(5044, 4) (624, 4) (631, 4)
